In [1]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

# 임베딩과 LLM모델 설정

In [2]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# Vector DB 생성

In [3]:
from langchain_chroma import Chroma

database = Chroma(
    collection_name='wine',
    persist_directory='./chroma_wine' 
    ,embedding_function=embedding
    )

# chat_template 설정

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to make each wine selection and pairing a memorable experience for users."
wine_query = "이 와인에 어울리는 요리에는 어떤 것들이 있을까요?"

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', SYSTEM_PROMPT),
        ('human', [{'type':'text', 'text' :'{text}'},
                   {'type':'image_url', 'image_url' : {'url' : '{image_url}'}},
                   ])
    ]
)

chat_template

ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to m

# recommend_dishes_chain 만들기

In [5]:
def recommend_dishes_chain(query):
    SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to make each wine selection and pairing a memorable experience for users."

    chat_template = ChatPromptTemplate.from_messages(
        [
            ('system', SYSTEM_PROMPT),
            ('human', [{'type':'text', 'text' :query['text']},
                    {'type':'image_url', 'image_url' : {'url' : query['image_url']}},
                    ])
        ]
    )
    chain = chat_template | llm  | StrOutputParser()
    return chain

In [6]:
query_1 = {
    'text': wine_query,
    'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}

rec_dish_chain = recommend_dishes_chain(query_1)

rec_dish_chain.invoke(query_1)


'Masserì Primitivo는 풍부하고 과일향이 나는 레드 와인으로, 다소 높은 알코올 농도와 탄닌 구조를 가지고 있어 다양한 음식과 잘 어울립니다. 아래는 이 와인과 잘 맞는 요리 목록입니다:\n\n1. **구운 고기 요리**: 양고기나 소고기 스테이크, 특히 바비큐 스타일로 조리한 고기와 잘 어울립니다.\n\n2. **이탈리안 파스타**: 미트 소스를 곁들인 파스타, 예를 들어 볼로네제 소스가 있는 라자냐나 스파게티와 좋습니다.\n\n3. **매운 요리**: 향신료가 들어간 고기 요리나 매운 소스를 곁들인 요리와도 잘 어우러집니다.\n\n4. **치즈**: 강한 풍미의 치즈, 예를 들어 블루치즈나 체다와 함께해보세요.\n\n5. **오븐에 구운 채소**: 기름진 채소를 구워서 곁들인 요리와도 잘 어울립니다.\n\n이 와인을 활용하여 요리를 선택하면 더욱 맛있는 식사를 즐길 수 있을 것입니다!'

In [7]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'Masserì Primitivo는 풍부한 과일 향과 부드러운 탄닌을 가진 레드 와인으로, 다양한 요리와 잘 어울립니다. 다음은 이 와인에 잘 맞는 요리 몇 가지입니다:\n\n1. **고기 요리**: 특히 구운 양고기나 소고기 스테이크와 잘 어울립니다. 고기의 풍미와 와인의 과일 향이 조화를 이룹니다.\n2. **이탈리안 요리**: 라자냐, 미트볼 파스타, 또는 리조또와 같은 진한 소스의 이탈리안 요리와 잘 맞습니다.\n3. **BBQ**: 바베큐 립이나 미트 번과 같은 그릴 요리도 좋은 선택입니다.\n4. **치즈**: 고다, 체다, 혹은 블루 치즈 같은 강한 풍미의 치즈와의 조화가 훌륭합니다.\n5. **탱커**: 올리브 오일을 뿌린 구운 채소나 버섯도 좋은 곁들임이 됩니다.\n\n그 외에도 깊은 풍미를 가진 요리와 잘 어울리니, 다양한 조합을 시도해보세요!'

# describe_dish_flavor_chain 프롬프트

In [8]:
def describe_dish_flavor_chain(query):
    SYSTEM_PROMPT = """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """

    chat_template = ChatPromptTemplate.from_messages(
        [
            ('system', SYSTEM_PROMPT),
            ('human', [{'type':'text', 'text' :query['text']},
                    {'type':'image_url', 'image_url' : {'url' : query['image_url']}},
                    ])
        ]
    )
    chain = chat_template | llm  | StrOutputParser()
    return chain

In [9]:
dish_query = '이 요리의 맛을 묘사해주세요 당신은 와인전문가이기도 하지만 미식평가단이기도 합니다'
query_2 = {
    'text': dish_query,
    'image_url': 'https://i2.wp.com/www.downshiftology.com/wp-content/uploads/2023/02/Filet-Mignon.jpg'
}

des_dish_chain = describe_dish_flavor_chain(query_2)

des_dish_chain.invoke(query_2)

'이 요리는 고급 스테이크에 허브 버터가 얹혀져 있는 모습으로, 눈으로 볼 때부터 질감과 맛의 조화가 기대됩니다.\n\n### 맛 분석\n- **주요 맛**: 스테이크의 풍부하고 진한 고기 맛이 가장 두드러집니다. 겉면은 고온에서 구워져 소금과 후추의 조화로운 크러스트를 형성하며, 이는 스테이크에 깊은 감칠맛을 더합니다. 스테이크의 내부는 부드럽고 육즙이 풍부하여, 한 입 베어물 때마다 고소한 풍미가 입안을 감돕니다.\n  \n- **허브 버터의 역할**: 허브 버터는 스테이크의 리치함을 강조하며, 신선한 허브의 향이 간혹 올라요. 버터의 크리미한 질감이 스테이크의 풍미를 부드럽게 감싸주며, 특히 파슬리와 같은 허브가 사용될 경우 약간의 상큼함과 신선함을 더해줍니다.\n\n### 질감과 향\n- **질감**: 스테이크의 겉은 바삭하고 마늘 같은 조리된 맛이 느껴지며, 속은 부드럽고 촉촉합니다. 허브 버터가 스테이크 위에 녹아내리며 크리미한 질감을 제공합니다.\n\n- **향**: 고깃결의 토대 위에 버터와 허브의 향이 어우러져 풍부하고 달콤한 향을 발산합니다. 구운 육류의 불향이 식욕을 자극합니다.\n\n### 음식과 음료의 조화\n이 요리는 풀 바디의 레드 와인과 잘 어울립니다. 특히, 까베르네 소비뇽 같은 강렬한 와인은 스테이크의 진한 맛과 상큼한 허브의 조합을 완벽히 보완합니다. 와인의 타닌이 고기의 기름과 조화를 이루어 고소함을 증폭시킵니다.\n\n결론적으로, 이 요리는 시각적으로도, 후각적으로도 매력을 느끼게 하며, 센세이션한 맛의 교향곡을 이루며 고급스러운 식사를 제공합니다.'

In [10]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./wine_data.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_data.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
variety: Portugu

In [11]:
from langchain_pinecone import PineconeVectorStore
import os

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding, 
    index_name='wine-reviews', 
    namespace='default'
)

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [12]:
results = vector_store.similarity_search(
    "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.", 
    k=5, 
    namespace='default'
)

results

[Document(id='78b33d63-6453-4489-9872-ea0c8fbb6c12', metadata={'row': 55680.0, 'source': './wine_data.csv'}, page_content=': 55680\ncountry: Italy\ndescription: Villa Novara presents easy aromas of fresh forest berry and almond paste. The wine is informal and sharp making it an ideal companion to pizza with mozzarella or pasta stuffed with ricotta cheese.\ndesignation: Villa Novare\npoints: 84\nprice: 16.0\nprovince: Veneto\nregion_1: Valpolicella Classico\nregion_2: \ntaster_name: \ntaster_twitter_handle: \ntitle: Bertani 2009 Villa Novare  (Valpolicella Classico)\nvariety: Corvina, Rondinella, Molinara\nwinery: Bertani'),
 Document(id='ef8d135c-c376-4d9a-8283-a39ab83f77a6', metadata={'row': 66281.0, 'source': './wine_data.csv'}, page_content=': 66281\ncountry: Italy\ndescription: A blend of 85% Sangiovese, 10% Alicante and 5% Ciliegiolo, this wine has earthy aromas that recall wet soil, leather and pressed blue flower petals. The juicy palate delivers fleshy black cherry and raspberr

# search_wine

In [13]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor,
        k=5,
        namespace='default'
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

In [14]:
taste_query = "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
runnable = RunnableLambda(search_wine)
response = runnable.invoke(taste_query)
print(response['dish_flavor'])
print(response['wine_reviews'])

이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.
: 55680
country: Italy
description: Villa Novara presents easy aromas of fresh forest berry and almond paste. The wine is informal and sharp making it an ideal companion to pizza with mozzarella or pasta stuffed with ricotta cheese.
designation: Villa Novare
points: 84
price: 16.0
province: Veneto
region_1: Valpolicella Classico
region_2: 
taster_name: 
taster_twitter_handle: 
title: Bertani 2009 Villa Novare  (Valpolicella Classico)
variety: Corvina, Rondinella, Molinara
winery: Bertani
: 66281
country: Italy
description: A blend of 85% Sangiovese, 10% Alicante and 5% Ciliegiolo, this wine has earthy aromas that recall wet soil, leather and pressed blue flower petals. The juicy palate delivers fleshy black cherry and raspberry. Notes of black pepper, espresso and Mediterranean herbs add interest.
designation: Ciabatta
points: 88
price: 25.0
province: Tuscany
region_1: Morellino di Scansano
region_2: 
taster_name: Kerin O

# recommend_wine

In [22]:
def recommend_wine(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona:

            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

            Role:

            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

            Examples:

            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
        """),
        ("human", """
            와인 페이링 추천에 아래 요리/맛, 와인 리뷰만을 참고하여 한글로 답변해 주시기 바랍니다.

            요리/맛:
            {dish_flavor}

            와인 리뷰:
            {wine_reviews}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    
    return chain

# chain의 연결

In [23]:
runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine)

chain = runnable_1 | runnable_2 | runnable_3

In [24]:
!pip install -qU grandalf

In [25]:
# 음식이미지 링크 > 음식찾아서 음식맛 
# > 음식맛이 와인평에 있는 와인 검색  
# > 와인평 + 음식맛 
# > 소믈리에메시지
chain.get_graph().print_ascii()

+----------------------------------+ 
| describe_dish_flavor_chain_input | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
          +-------------+            
          | search_wine |            
          +-------------+            
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
      +-----------------------+      
      | recommend_wine_output |      
      +-----

In [26]:
response = chain.invoke(query_2)

response

'이 프라임 스테이크에 허브 버터가 얹혀진 요리는 깊고 복합적인 풍미가 특징으로, 함께 곁들여진 아루굴라의 신선함이 전체적인 식사를 더욱 풍성하게 해줍니다. 이런 요리에는 풀 바디의 레드 와인이 알맞습니다. \n\n추천드릴 와인은 **Magnificent Wine Company 2010 Steak House Cabernet Sauvignon (Columbia Valley)**입니다. 이 와인은 카베르네 소비뇽으로, 와인의 탄닌이 스테이크의 기름진 맛과 잘 어우러져 조화를 이루고, 고기의 육즙과 허브의 향을 한층 강조해줍니다. 와인은 섬세하게 구운 향과 미세한 체리 및 자두의 맛이 어우러져 부드러운 질감을 선사하여, 스테이크와 함께 즐기기에 적합합니다. 이 와인은 가격대도 합리적이며(10달러), 품질 면에서도 우수함을 자랑합니다.\n\n또한, **Hartenberg 2007 Cabernet Sauvignon (Stellenbosch)**도 추천드립니다. 이 와인은 허브의 노트와 잘 익은 블랙과일 맛이 균형을 이루면서, 달콤한 바닐라의 뉘앙스가 더해져 풍부한 맛을 가지고 있습니다. 전체적인 조화가 뛰어나며, 스테이크와 허브 버터의 조합과 환상적인 페어링을 만들어 줄 것입니다. 가격은 35달러로, 품질에 비해 좋은 선택이 될 수 있습니다.\n\n이 두 가지 와인 모두 이 맛있는 스테이크 요리와 잘 어울리며, 더 깊고 특별한 미식 경험을 제공할 것입니다.'